In [63]:
import torch
import numpy as np

In [39]:
# load multigraph

path = "./Data/multigraphs/heteroData_gene_cell_Neuroblastoma_cgp_cnv_META.pt"

new_data = torch.load(path)
print(new_data)


HeteroData(
  metapath_dict={
    (gene, metapath_0, gene)=[2],
    (cell, metapath_1, cell)=[2],
  },
  gene={
    node_id=[13398],
    names=[13398],
    x=[13398, 3438],
  },
  cell={
    node_id=[37],
    names=[37],
    x=[37, 1218],
  },
  (gene, interacts_with, gene)={ edge_index=[2, 263122] },
  (gene, dependency_of, cell)={ edge_index=[2, 14085] },
  (gene, rev_interacts_with, gene)={ edge_index=[2, 263122] },
  (cell, rev_dependency_of, gene)={ edge_index=[2, 14085] },
  (gene, metapath_0, gene)={
    edge_index=[2, 719373],
    edge_weight=[719373],
  },
  (cell, metapath_1, cell)={
    edge_index=[2, 1369],
    edge_weight=[1369],
  }
)


In [18]:
path = "./Data/multigraphs/heteroData_gene_cell_Neuroblastoma_Reactome_crispr-1_5_cgp_cnv.pt"

data_old = torch.load(path)

data_old

HeteroData(
  gene={
    node_id=[14034],
    names=[14034],
    x=[14034, 3438],
  },
  cell={
    node_id=[37],
    names=[37],
    x=[37, 1218],
  },
  (gene, interacts_with, gene)={ edge_index=[2, 278974] },
  (gene, dependency_of, cell)={ edge_index=[2, 14099] },
  (gene, rev_interacts_with, gene)={ edge_index=[2, 278974] },
  (cell, rev_dependency_of, gene)={ edge_index=[2, 14099] }
)

In [40]:
import numpy as np
print(np.__version__)

2.1.2


The difference between these two graphs is that in the first graph I removed the common essential genes, whereas in the secon graph I kept them. Therefore the second graph has many more dependencies.

In [19]:
print(new_data)
print(data_old)

HeteroData(
  metapath_dict={
    (gene, metapath_0, gene)=[2],
    (cell, metapath_1, cell)=[2],
  },
  gene={
    node_id=[13398],
    names=[13398],
    x=[13398, 3438],
  },
  cell={
    node_id=[37],
    names=[37],
    x=[37, 1218],
  },
  (gene, interacts_with, gene)={ edge_index=[2, 263122] },
  (gene, dependency_of, cell)={ edge_index=[2, 14085] },
  (gene, rev_interacts_with, gene)={ edge_index=[2, 263122] },
  (cell, rev_dependency_of, gene)={ edge_index=[2, 14085] },
  (gene, metapath_0, gene)={
    edge_index=[2, 719373],
    edge_weight=[719373],
  },
  (cell, metapath_1, cell)={
    edge_index=[2, 1369],
    edge_weight=[1369],
  }
)
HeteroData(
  gene={
    node_id=[14034],
    names=[14034],
    x=[14034, 3438],
  },
  cell={
    node_id=[37],
    names=[37],
    x=[37, 1218],
  },
  (gene, interacts_with, gene)={ edge_index=[2, 278974] },
  (gene, dependency_of, cell)={ edge_index=[2, 14099] },
  (gene, rev_interacts_with, gene)={ edge_index=[2, 278974] },
  (cell, re

In [37]:
import torch
from collections import defaultdict

def enhanced_compare_heterodata(hdata1, hdata2):
    """
    Compare two PyTorch Geometric HeteroData objects in detail,
    checking node name-ID mappings and exact edge consistency.
    """
    print("Comparing node types:")
    node_types1 = set(hdata1.node_types)
    node_types2 = set(hdata2.node_types)
    common_node_types = node_types1 & node_types2
    print("  Common node types:", common_node_types)
    
    # Dictionary to store node_id to name mappings for both graphs
    id_to_name1 = defaultdict(dict)
    id_to_name2 = defaultdict(dict)
    name_to_id1 = defaultdict(dict)
    name_to_id2 = defaultdict(dict)
    
    # First, extract node mappings (ID to name and name to ID)
    for nt in common_node_types:
        if hasattr(hdata1[nt], "node_id") and hasattr(hdata1[nt], "names") and \
           hasattr(hdata2[nt], "node_id") and hasattr(hdata2[nt], "names"):
            
            # Extract IDs and names
            ids1 = hdata1[nt].node_id
            names1 = hdata1[nt].names
            ids2 = hdata2[nt].node_id
            names2 = hdata2[nt].names
            
            # Convert to Python lists if they're tensors
            ids1_list = ids1.tolist() if isinstance(ids1, torch.Tensor) else ids1
            names1_list = names1.tolist() if isinstance(names1, torch.Tensor) else names1
            ids2_list = ids2.tolist() if isinstance(ids2, torch.Tensor) else ids2
            names2_list = names2.tolist() if isinstance(names2, torch.Tensor) else names2
            
            # Create mappings
            for i, name in zip(ids1_list, names1_list):
                id_to_name1[nt][i] = name
                name_to_id1[nt][name] = i
                
            for i, name in zip(ids2_list, names2_list):
                id_to_name2[nt][i] = name
                name_to_id2[nt][name] = i
            
            # Compare by names instead of IDs
            names1_set = set(names1_list)
            names2_set = set(names2_list)
            overlap = names1_set & names2_set
            
            print(f"  Node type '{nt}':")
            print(f"    hdata1 has {len(names1_set)} nodes, hdata2 has {len(names2_set)} nodes")
            print(f"    Overlap by name: {len(overlap)} nodes ({len(overlap)/len(names1_set)*100:.2f}% of hdata1, {len(overlap)/len(names2_set)*100:.2f}% of hdata2)")
            
            # Check if name-ID mappings are consistent
            inconsistent_mappings = []
            for name in overlap:
                id1 = name_to_id1[nt][name]
                id2 = name_to_id2[nt][name]
                if id1 != id2:
                    inconsistent_mappings.append((name, id1, id2))
            
            if inconsistent_mappings:
                print(f"    Found {len(inconsistent_mappings)} inconsistent name-ID mappings")
                print(f"    First 5 inconsistencies: {inconsistent_mappings[:5]}")
            else:
                print("    All name-ID mappings are consistent between graphs")
        else:
            print(f"  Node type '{nt}' is missing node_id or names attributes in one or both graphs")
    
    print("\nComparing edge types:")
    edge_types1 = set(hdata1.edge_types) if hasattr(hdata1, "edge_types") else set(hdata1.keys()) - set(hdata1.node_types)
    edge_types2 = set(hdata2.edge_types) if hasattr(hdata2, "edge_types") else set(hdata2.keys()) - set(hdata2.node_types)
    common_edge_types = edge_types1 & edge_types2
    print("  Common edge types:", common_edge_types)
    
    for et in common_edge_types:
        src_type, edge_type, dst_type = et
        
        if hasattr(hdata1[et], "edge_index") and hasattr(hdata2[et], "edge_index"):
            edges1 = hdata1[et].edge_index
            edges2 = hdata2[et].edge_index
            
            num_edges1 = edges1.shape[1] if isinstance(edges1, torch.Tensor) else len(edges1)
            num_edges2 = edges2.shape[1] if isinstance(edges2, torch.Tensor) else len(edges2)
            
            print(f"\n  Edge type {et}:")
            print(f"    hdata1 has {num_edges1} edges, hdata2 has {num_edges2} edges")
            
            # Convert edge indices to edges with names
            if (src_type in name_to_id1 and dst_type in name_to_id1 and 
                src_type in name_to_id2 and dst_type in name_to_id2):
                
                # Convert edge indices to lists
                edges1_list = edges1.t().tolist() if isinstance(edges1, torch.Tensor) else edges1
                edges2_list = edges2.t().tolist() if isinstance(edges2, torch.Tensor) else edges2
                
                # Convert node IDs to node names for edges
                named_edges1 = set()
                for src_id, dst_id in edges1_list:
                    if src_id in id_to_name1[src_type] and dst_id in id_to_name1[dst_type]:
                        src_name = id_to_name1[src_type][src_id]
                        dst_name = id_to_name1[dst_type][dst_id]
                        named_edges1.add((src_name, dst_name))
                    
                named_edges2 = set()
                for src_id, dst_id in edges2_list:
                    if src_id in id_to_name2[src_type] and dst_id in id_to_name2[dst_type]:
                        src_name = id_to_name2[src_type][src_id]
                        dst_name = id_to_name2[dst_type][dst_id]
                        named_edges2.add((src_name, dst_name))
                
                # Compare exact edges by name
                common_edges = named_edges1 & named_edges2
                only_in_1 = named_edges1 - named_edges2
                only_in_2 = named_edges2 - named_edges1
                
                print(f"    Comparing edges by node names:")
                print(f"      Total edges by name in hdata1: {len(named_edges1)}")
                print(f"      Total edges by name in hdata2: {len(named_edges2)}")
                print(f"      Common edges: {len(common_edges)} ({len(common_edges)/len(named_edges1)*100:.2f}% of hdata1, {len(common_edges)/len(named_edges2)*100:.2f}% of hdata2)")
                print(f"      Edges only in hdata1: {len(only_in_1)}")
                print(f"      Edges only in hdata2: {len(only_in_2)}")
                
                if len(only_in_1) > 0:
                    print(f"      First 3 sample edges only in hdata1: {list(only_in_1)[:3]}")
                if len(only_in_2) > 0:
                    print(f"      First 3 sample edges only in hdata2: {list(only_in_2)[:3]}")
            else:
                print(f"    Cannot compare edges by name for {et} due to missing name mappings")
        else:
            print(f"  Edge type {et}: Edge index attribute not found in one of the objects")
            
    return id_to_name1, name_to_id1, id_to_name2, name_to_id2


def print_node_mappings(hdata, node_type, limit=10):
    """
    Print the node ID to name mappings for a specific node type in a HeteroData object.
    
    Args:
        hdata: PyTorch Geometric HeteroData object
        node_type: The node type to print mappings for (e.g., 'gene', 'cell')
        limit: Maximum number of mappings to print
    """
    if not hasattr(hdata[node_type], "node_id") or not hasattr(hdata[node_type], "names"):
        print(f"Node type '{node_type}' is missing node_id or names attributes")
        return
    
    node_ids = hdata[node_type].node_id
    names = hdata[node_type].names
    
    # Convert to Python lists if they're tensors
    node_ids_list = node_ids.tolist() if isinstance(node_ids, torch.Tensor) else node_ids
    names_list = names.tolist() if isinstance(names, torch.Tensor) else names
    
    print(f"\nNode mappings for '{node_type}':")
    print(f"Total nodes: {len(node_ids_list)}")
    
    print("\nSample node_id to name mappings:")
    for i, (node_id, name) in enumerate(zip(node_ids_list, names_list)):
        if i >= limit:
            break
        print(f"  {node_id} -> {name}")
    
    # Check for duplicates
    unique_ids = set(node_ids_list)
    unique_names = set(names_list)
    
    print(f"\nUnique IDs: {len(unique_ids)} (Expected: {len(node_ids_list)})")
    print(f"Unique names: {len(unique_names)} (Expected: {len(names_list)})")
    
    if len(unique_ids) < len(node_ids_list):
        print("WARNING: Duplicate node IDs detected!")
    
    if len(unique_names) < len(names_list):
        print("WARNING: Duplicate node names detected!")


def find_node_by_name(hdata, node_type, name_pattern):
    """
    Find nodes in a HeteroData object where the name contains the given pattern.
    
    Args:
        hdata: PyTorch Geometric HeteroData object
        node_type: The node type to search in (e.g., 'gene', 'cell')
        name_pattern: A string pattern to search for in node names
    """
    if not hasattr(hdata[node_type], "node_id") or not hasattr(hdata[node_type], "names"):
        print(f"Node type '{node_type}' is missing node_id or names attributes")
        return []
    
    node_ids = hdata[node_type].node_id
    names = hdata[node_type].names
    
    # Convert to Python lists if they're tensors
    node_ids_list = node_ids.tolist() if isinstance(node_ids, torch.Tensor) else node_ids
    names_list = names.tolist() if isinstance(names, torch.Tensor) else names
    
    matching_nodes = []
    for node_id, name in zip(node_ids_list, names_list):
        if name_pattern.lower() in str(name).lower():
            matching_nodes.append((node_id, name))
    
    print(f"Found {len(matching_nodes)} {node_type} nodes matching '{name_pattern}':")
    for i, (node_id, name) in enumerate(matching_nodes[:10]):
        print(f"  {node_id} -> {name}")
    
    if len(matching_nodes) > 10:
        print(f"  ... and {len(matching_nodes) - 10} more")
    
    return matching_nodes


def compare_nodes_between_graphs(hdata1, hdata2, node_type):
    """
    Compare nodes between two HeteroData objects for a specific node type.
    
    Args:
        hdata1: First PyTorch Geometric HeteroData object
        hdata2: Second PyTorch Geometric HeteroData object
        node_type: The node type to compare (e.g., 'gene', 'cell')
    """
    if (not hasattr(hdata1[node_type], "node_id") or not hasattr(hdata1[node_type], "names") or 
        not hasattr(hdata2[node_type], "node_id") or not hasattr(hdata2[node_type], "names")):
        print(f"Node type '{node_type}' is missing node_id or names attributes in one or both graphs")
        return
    
    # Extract IDs and names
    ids1 = hdata1[node_type].node_id
    names1 = hdata1[node_type].names
    ids2 = hdata2[node_type].node_id
    names2 = hdata2[node_type].names
    
    # Convert to Python lists if they're tensors
    ids1_list = ids1.tolist() if isinstance(ids1, torch.Tensor) else ids1
    names1_list = names1.tolist() if isinstance(names1, torch.Tensor) else names1
    ids2_list = ids2.tolist() if isinstance(ids2, torch.Tensor) else ids2
    names2_list = names2.tolist() if isinstance(names2, torch.Tensor) else names2
    
    # Create name sets for comparison
    names1_set = set(names1_list)
    names2_set = set(names2_list)
    
    common_names = names1_set & names2_set
    only_in_1 = names1_set - names2_set
    only_in_2 = names2_set - names1_set
    
    print(f"\nComparing {node_type} nodes between graphs:")
    print(f"  Graph 1: {len(names1_set)} unique nodes")
    print(f"  Graph 2: {len(names2_set)} unique nodes")
    print(f"  Common nodes: {len(common_names)} ({len(common_names)/len(names1_set)*100:.2f}% of graph 1, {len(common_names)/len(names2_set)*100:.2f}% of graph 2)")
    print(f"  Nodes only in graph 1: {len(only_in_1)}")
    print(f"  Nodes only in graph 2: {len(only_in_2)}")
    
    if len(only_in_1) > 0:
        print(f"  Sample nodes only in graph 1: {list(only_in_1)[:5]}")
    if len(only_in_2) > 0:
        print(f"  Sample nodes only in graph 2: {list(only_in_2)[:5]}")
    
    # Create mappings for ID consistency check
    name_to_id1 = {name: id for id, name in zip(ids1_list, names1_list)}
    name_to_id2 = {name: id for id, name in zip(ids2_list, names2_list)}
    
    # Check for ID consistency for common names
    inconsistent = []
    for name in common_names:
        if name_to_id1[name] != name_to_id2[name]:
            inconsistent.append((name, name_to_id1[name], name_to_id2[name]))
    
    if inconsistent:
        print(f"\n  Found {len(inconsistent)} nodes with inconsistent IDs between graphs")
        print(f"  Sample inconsistencies (name, id1, id2): {inconsistent[:5]}")
    else:
        print("\n  All common nodes have consistent IDs between graphs")


# Example usage:
id_to_name1, name_to_id1, id_to_name2, name_to_id2 = enhanced_compare_heterodata(new_data, data_old)
print_node_mappings(new_data, 'gene')
print_node_mappings(data_old, 'cell')
find_node_by_name(new_data, 'gene', 'BRCA')
compare_nodes_between_graphs(new_data, data_old, 'gene')

Comparing node types:
  Common node types: {'gene', 'cell'}
  Node type 'gene':
    hdata1 has 13398 nodes, hdata2 has 14034 nodes
    Overlap by name: 13398 nodes (100.00% of hdata1, 95.47% of hdata2)
    Found 13393 inconsistent name-ID mappings
    First 5 inconsistencies: [(np.str_('DCUN1D4'), 2687, 2730), (np.str_('ZNF550'), 13154, 13769), (np.str_('PDE6H'), 8290, 8720), (np.str_('RPUSD4'), 9935, 10402), (np.str_('FOXE3'), 3869, 3993)]
  Node type 'cell':
    hdata1 has 37 nodes, hdata2 has 37 nodes
    Overlap by name: 37 nodes (100.00% of hdata1, 100.00% of hdata2)
    All name-ID mappings are consistent between graphs

Comparing edge types:
  Common edge types: {('gene', 'interacts_with', 'gene'), ('gene', 'dependency_of', 'cell'), ('gene', 'rev_interacts_with', 'gene'), ('cell', 'rev_dependency_of', 'gene')}

  Edge type ('gene', 'interacts_with', 'gene'):
    hdata1 has 263122 edges, hdata2 has 278974 edges
    Comparing edges by node names:
      Total edges by name in hdata

In [61]:

path = "./Data/multigraphs/heteroData_gene_cell_All_cgp_cnv_META.pt"

new_data = torch.load(path)
print(new_data)

TypeError: scalar() argument 1 must be numpy.dtype, not numpy.dtypes.StrDType

In [66]:
#print numpy version
print(np.__version__)

2.1.2


In [65]:
import sys
print(sys.executable)

/Users/jovanafilipovic/Downloads/MSc Bioinformatics/Year 2/Thesis/GIT_HetGNN/env/bin/python


In [56]:
!pip list | grep numpy
!pip list | grep torch

numpy              1.25.1
torch              2.3.1
torch_cluster      1.6.3
torch-geometric    2.6.1
torch_scatter      2.1.2
torch_sparse       0.6.18
torch_spline_conv  1.2.2
torchvision        0.18.1
